# Preprocessing Using Dataflow 

**Learning Objectives**
- Creating datasets for Machine Learning using Dataflow

## Introduction 

While Pandas is fine for experimenting, for operationalization of your workflow, it is better to do preprocessing in Apache Beam. This will also help if you need to preprocess data in flight, since Apache Beam also allows for streaming.

Note that Apache Beam only works in Python 2 at the moment, so we'll be using a Python 2 kernel. Check that you are running a Python 2 kernel in the upper right corner of this notebook. If necessary, click there and in the dropdown menu and select `python2`. 

Once we have activated a Python 2 environment, we'll install Apache Beam and the necessary libraries.

### Set environment variables and load necessary libraries
Execute the following cells to install the necessary libraries if they have not been installed already.

In [1]:
# Ensure that we have Tensorflow 1.13 installed.
!pip freeze | grep tensorflow==1.13.1 || pip install tensorflow==1.13.1

tensorflow==1.13.1


In [ ]:
#Ensure that we have Apache Beam 2.10 installed.
!pip freeze | grep apache-beam==2.10.0 || pip install apache-beam[gcp]

In [1]:
import tensorflow as tf
import apache_beam as beam
import shutil
import os
print(tf.__version__)

1.13.1


Next, set the environment variables related to your GCP Project.

In [2]:
PROJECT = "cloud-training-demos"  # Replace with your PROJECT
BUCKET = "cloud-training-bucket"  # Replace with your BUCKET
REGION = "us-central1"            # Choose an available region for Cloud MLE
TFVERSION = "1.13"                # TF version for CMLE to use

In [3]:
import os
os.environ["BUCKET"] = BUCKET
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [4]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Save the query from earlier

The data is natality data (record of births in the US). My goal is to predict the baby's weight given a number of factors about the pregnancy and the baby's mother.  Later, we will want to split the data into training and eval datasets. The hash of the year-month will be used for that.

In [5]:
# Create SQL query using natality data after the year 2000
query_string = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    year > 2000
"""

In [6]:
# Call BigQuery and examine in dataframe
from google.cloud import bigquery
bq = bigquery.Client(project = PROJECT)

df = bq.query(query_string + "LIMIT 100").to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.936641,False,30,1,38,6691862025345277042
1,8.624484,True,34,1,39,7170969733900686954
2,6.937947,True,23,1,39,7170969733900686954
3,6.935743,True,18,1,39,1525201076796226340
4,8.318041,False,23,1,40,8904940584331855459


## Create ML dataset using Dataflow

Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

Instead of using Beam/Dataflow, I had three other options:

* Use Cloud Dataprep to visually author a Dataflow pipeline. Cloud Dataprep also allows me to explore the data, so we could have avoided much of the handcoding of Python/Seaborn calls above as well!
* Read from BigQuery directly using TensorFlow.
* Use the BigQuery console (http://bigquery.cloud.google.com) to run a Query and save the result as a CSV file. For larger datasets, you may have to select the option to "allow large results" and save the result into a CSV file on Google Cloud Storage. 

However, in this case, I want to do some preprocessing, modifying data so that we can simulate what is known if no ultrasound has been performed. If I didn't need preprocessing, I could have used the web console. Also, I prefer to script it out rather than run queries on the user interface, so I am using Cloud Dataflow for the preprocessing.

The `preprocess` function below includes an arugment `in_test_mode`. When this is set to `True`, running `preprocess` initiates a *local* Beam job. This is helpful for quickly debugging your pipeline and ensuring it works before submitting a job to the Cloud.  Setting `in_test_mode` to `False` will launch a processing that is happening on the Cloud. Go to the GCP webconsole to [the Dataflow section](https://pantheon.corp.google.com/dataflow) and monitor the running job. It took about 20 minutes for me.

If you wish to continue without doing this step, you can copy my preprocessed output:
<pre>
gsutil -m cp -r gs://cloud-training-demos/babyweight/preproc gs://YOUR_BUCKET/
</pre>

In [8]:
import apache_beam as beam
import datetime, os

def to_csv(rowdict):
    # Pull columns from BQ and create a line
    import hashlib
    import copy
    CSV_COLUMNS = "weight_pounds,is_male,mother_age,plurality,gestation_weeks".split(',')

    # Create synthetic data where we assume that no ultrasound has been performed
    # and so we don"t know sex of the baby. Let"s assume that we can tell the difference
    # between single and multiple, but that the errors rates in determining exact number
    # is difficult in the absence of an ultrasound.
    no_ultrasound = copy.deepcopy(rowdict)
    w_ultrasound = copy.deepcopy(rowdict)

    no_ultrasound["is_male"] = "Unknown"
    if rowdict["plurality"] > 1:
        no_ultrasound["plurality"] = "Multiple(2+)"
    else:
        no_ultrasound["plurality"] = "Single(1)"

    # Change the plurality column to strings
    w_ultrasound["plurality"] = ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"][rowdict["plurality"] - 1]

    # Write out two rows for each input row, one with ultrasound and one without
    for result in [no_ultrasound, w_ultrasound]:
        data = ','.join([str(result[k]) if k in result else "None" for k in CSV_COLUMNS])
        key = hashlib.sha224(data).hexdigest()  # hash the columns to form a key
        yield str("{},{}".format(data, key))
  
def preprocess(in_test_mode):
    import shutil, os, subprocess
    job_name = "preprocess-babyweight-features" + "-" + datetime.datetime.now().strftime("%y%m%d-%H%M%S")

    if in_test_mode:
        print("Launching local job ... hang on")
        OUTPUT_DIR = "./preproc"
        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        os.makedirs(OUTPUT_DIR)
    else:
        print("Launching Dataflow job {} ... hang on".format(job_name))
        OUTPUT_DIR = "gs://{0}/babyweight/preproc/".format(BUCKET)
        try:
            subprocess.check_call("gsutil -m rm -r {}".format(OUTPUT_DIR).split())
        except:
            pass

    options = {
        "staging_location": os.path.join(OUTPUT_DIR, "tmp", "staging"),
        "temp_location": os.path.join(OUTPUT_DIR, "tmp"),
        "job_name": job_name,
        "project": PROJECT,
        "teardown_policy": "TEARDOWN_ALWAYS",
        "no_save_main_session": True
    }
    opts = beam.pipeline.PipelineOptions(flags = [], **options)
    if in_test_mode:
        RUNNER = "DirectRunner"
    else:
        RUNNER = "DataflowRunner"
        
    p = beam.Pipeline(RUNNER, options = opts)
    query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
    publicdata.samples.natality
WHERE
    year > 2000
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
    AND month > 0
"""

    if in_test_mode:
        query = query + " LIMIT 100" 

    for step in ["train", "eval"]:
        if step == "train":
            selquery = "SELECT * FROM ({}) WHERE MOD(hashmonth, 100) < 80".format(query)
        elif step == "eval":
            selquery = "SELECT * FROM ({}) WHERE MOD(hashmonth, 100) >= 80 AND MOD(hashmonth, 100) < 90".format(query)
        else: 
            selquery = "SELECT * FROM ({}) WHERE MOD(hashmonth, 100) >= 90".format(query)
        (p 
         | "{}_read".format(step) >> beam.io.Read(beam.io.BigQuerySource(query = selquery, use_standard_sql = True))
         | "{}_csv".format(step) >> beam.FlatMap(to_csv)
         | "{}_out".format(step) >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, "{}.csv".format(step))))
        )

    job = p.run()
    if in_test_mode:
        job.wait_until_finish()
        print("Done!")
    
preprocess(in_test_mode = False)

Launching Dataflow job preprocess-babyweight-features-190409-204323 ... hang on


For a Cloud preprocessing job (i.e. setting `in_test_mode` to `False`), the above step will take 20+ minutes. Go to the GCP web console, navigate to the Dataflow section and <b>wait for the job to finish</b> before you run the follwing step.

## View results
We can have a look at the elements in our bucket to see the results of our pipeline above.

In [ ]:
!gsutil ls gs://$BUCKET/babyweight/preproc/*-00000*

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License